In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [3]:
from config import list_cols_with_na

In [77]:
# Reading data
# Train
df_train = pd.read_csv("./data/train.csv")
# Test
# Note that test set has no SalePrice, which is what you are going to predict
# Therefore, you need to get your own "test" set from the train data to evaluate your model
# The test set is evaluated onto kaggle
df_test = pd.read_csv("./data/test.csv")

# Shapes of dataframes
print(df_train.shape)
print(df_test.shape)

(1460, 81)
(1459, 80)


0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64

In [5]:
df_train_copy = df_train.copy()
df_train_copy["tt"] = "train"
df_test_copy = df_test.copy()
df_test_copy["tt"] = "test"
df = pd.concat([df_train_copy, df_test_copy]).reset_index(drop=True)

In [82]:
# Print columns with missing values
missing_cols = df.isna().sum().loc[(df.isna().sum() > 0)]
print(df[list(missing_cols.loc[~missing_cols.index.isin(list_cols_with_na)].index)].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MSZoning      2915 non-null   object 
 1   LotFrontage   2433 non-null   float64
 2   Utilities     2917 non-null   object 
 3   Exterior1st   2918 non-null   object 
 4   Exterior2nd   2918 non-null   object 
 5   MasVnrArea    2896 non-null   float64
 6   BsmtFinSF1    2918 non-null   float64
 7   BsmtFinSF2    2918 non-null   float64
 8   BsmtUnfSF     2918 non-null   float64
 9   TotalBsmtSF   2918 non-null   float64
 10  Electrical    2918 non-null   object 
 11  BsmtFullBath  2917 non-null   float64
 12  BsmtHalfBath  2917 non-null   float64
 13  KitchenQual   2918 non-null   object 
 14  Functional    2917 non-null   object 
 15  GarageYrBlt   2760 non-null   float64
 16  GarageCars    2918 non-null   float64
 17  GarageArea    2918 non-null   float64
 18  SaleType      2918 non-null 

### The columns above that contain missing values without explanation
### The columns mentioned below will be cleaned through rules

### All categorical columns above will be imputed with the most frequent, LotFrontage will be imputed through SVR

In [11]:
# Check if fireplace quality matches fireplaces
print(df.loc[df["FireplaceQu"].isna()]["Fireplaces"].describe())

count    1420.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: Fireplaces, dtype: float64


In [14]:
# Check BsmtFinSF1 and BsmtFinSF2 values when there are no basements
print(df.loc[df["BsmtFinType1"].isna()]["BsmtFinSF1"].describe())

count    78.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
Name: BsmtFinSF1, dtype: float64


In [28]:
# Check basement data
# "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinSF1", "BsmtFinType2", "BsmtFinSF2", "TotalBsmtSF", "BsmtUnfSF"
df.loc[(df[["BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinSF1", "BsmtFinType2", "BsmtFinSF2", "TotalBsmtSF"]].isna().any(axis=1)
        & (df["BsmtFinSF1"] != 0))][["BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinSF1", "BsmtUnfSF", "BsmtFinType2", "BsmtFinSF2", "TotalBsmtSF"]]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtUnfSF,BsmtFinType2,BsmtFinSF2,TotalBsmtSF
332,Gd,TA,No,GLQ,1124.0,1603.0,NaN,479.0,3206.0
2040,Gd,NaN,Mn,GLQ,1044.0,0.0,Rec,382.0,1426.0
2120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2185,TA,NaN,No,BLQ,1033.0,94.0,Unf,0.0,1127.0
2524,TA,NaN,Av,ALQ,755.0,240.0,Unf,0.0,995.0


In [23]:
# Check MasVnr
# MasVnrArea should be zero when MasVnrType is NA
df.loc[(df["MasVnrType"].isna()) & (df["MasVnrArea"] != 0)][["MasVnrType", "MasVnrArea"]]

,MasVnrType,MasVnrArea
234,NaN,NaN
529,NaN,NaN
624,NaN,288.0
650,NaN,NaN
773,NaN,1.0
936,NaN,NaN
973,NaN,NaN
977,NaN,NaN
1230,NaN,1.0
1243,NaN,NaN


In [30]:
# BsmtFullBath
# BsmtFullBath and BsmtHalfBath should be zero when BsmtQual is NA
df.loc[df["BsmtQual"].isna(), ["BsmtFullBath", "BsmtHalfBath"]].describe()

,BsmtFullBath,BsmtHalfBath
count,79.0,79.0
mean,0.0,0.0
std,0.0,0.0
min,0.0,0.0
25%,0.0,0.0
50%,0.0,0.0
75%,0.0,0.0
max,0.0,0.0


In [72]:
# Check garage
# If GarageType is NA, GarageCars and GarageArea must be zero
temp_df = (df.loc[(df["GarageType"].isna()), ["GarageYrBlt", "GarageFinish", "GarageCars", "GarageArea","GarageQual", "GarageCond"]])
print(temp_df.isna().sum())
print(temp_df.shape)

GarageYrBlt     157
GarageFinish    157
GarageCars        0
GarageArea        0
GarageQual      157
GarageCond      157
dtype: int64
(157, 6)
